In [1]:
#JJ

In [3]:
# for colab run
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/projeto/automl-dynamic-search-space/analysis/autosklearn-dss/")

Mounted at /content/drive


In [6]:
# os.listdir()

In [5]:
# !unzip auto-results-3600.zip
# !unzip auto-results_v2.zip
# !unzip auto-results_v3.zip
# !unzip auto-results_v4.zip
# !unzip auto-results-space.zip

## Search Space

In [9]:
# os.listdir("auto-results-space/autosklearn_dss_90_mtl_space/")

In [29]:
import json
import pandas as pd

def get_all_csv_files(root_folder, how):
    csv_files = []
    for dirpath, _, filenames in os.walk(root_folder):
        for file in filenames:
            if file.lower().endswith(how):
                csv_files.append(os.path.join(dirpath, file))
    return csv_files

In [30]:
root_folder = "auto-results-space/autosklearn_dss_90_mtl_space/"

json_files = get_all_csv_files(root_folder, ".json")
json_obj = [json.load(open(p, "r")) for p in json_files]
datasets = ["dataset_" + f.split("/")[-1].split("_")[1] for f in json_files]
json_obj =  [{"classifier": len(d["classifier"]), "feature_preprocessor": len(d["feature_preprocessor"]), "dataset": idx} for d, idx in zip(json_obj, datasets)]
df_space = pd.DataFrame(json_obj)
df_space

,classifier,feature_preprocessor,dataset
0,6,8,dataset_40985
1,6,8,dataset_40985
2,6,8,dataset_40985
3,6,8,dataset_40985
4,6,8,dataset_40985
...,...,...,...
395,8,6,dataset_816
396,8,6,dataset_816
397,8,6,dataset_816
398,8,6,dataset_816


In [40]:
df_space\
  .groupby(["dataset"])\
  .mean()\
  .agg(
      mean_classifier=("classifier", "mean"),
      std_classifier=("classifier", "std"),
      mean_feature_preprocessor=("feature_preprocessor", "mean"),
      std_feature_preprocessor=("feature_preprocessor", "std")
  )

,classifier,feature_preprocessor
mean_classifier,6.500000,NaN
std_classifier,1.377474,NaN
mean_feature_preprocessor,NaN,7.075000
std_feature_preprocessor,NaN,0.916725


## Performance Evaluation

In [47]:
import os
import pandas as pd

how = "performance.csv"
root_path = "auto-results_v4/"
csv_files = get_all_csv_files(root_path, how)
result_v4 = [pd.read_csv(c).assign(seed=c.split("/")[-2]) for c in csv_files]

In [49]:
import os
import pandas as pd


how = "train_val_test_result.csv"
root_path = "auto-results-3600/"
csv_files = get_all_csv_files(root_path, how)
datasets = ["dataset_" + f.split("/")[-1].split("_")[1] for f in csv_files]
methods = ["_".join(f.split("/")[-1].split("_")[2:4]) for f in csv_files]
methods = ["autosklearn" if f == "autosklearn_cv" else f for f in methods]
seeds = [c.split("/")[-2] for c in csv_files]

result = []
for m, d, c, s in zip(methods, datasets, csv_files, seeds):
  aux = pd.read_csv(c)
  aux["method"] = m
  aux["dataset"] = d
  aux["seed"] = s
  result.append(aux)

In [131]:
df_3600 = pd.concat(result).query("method == 'autosklearn' or method == 'random_forest' ")
for r, i in zip(df_3600.iterrows(), range(len(df_3600))):
  if r[1]["method"] == "autosklearn":
    df_3600.iloc[i, 3] = "autosklearn_mtl"

df_v4 = pd.concat(result_v4).query("method == 'autosklearn' or method == 'autosklearn_dss_90' or method == 'autosklearn_dss_95' or method == 'autosklearn_dss_90_mtl'")

df = pd.concat([df_3600, df_v4]).sort_values(["method", "dataset", "seed"])
df

,train,val,test,method,dataset,seed
0,0.934680,0.804797,0.773813,autosklearn,dataset_1044,0
0,0.919859,0.758528,0.738648,autosklearn,dataset_1044,1
0,0.917721,0.753187,0.720948,autosklearn,dataset_1044,2
0,0.921352,0.762689,0.750045,autosklearn,dataset_1044,3
0,0.929193,0.787933,0.779248,autosklearn,dataset_1044,4
...,...,...,...,...,...,...
0,1.000000,1.000000,1.000000,random_forest,dataset_951,5
0,1.000000,1.000000,1.000000,random_forest,dataset_951,6
0,1.000000,1.000000,1.000000,random_forest,dataset_951,7
0,1.000000,1.000000,1.000000,random_forest,dataset_951,8


In [132]:
df.groupby(["method"])["method"].count()

,method
method,
autosklearn,400
autosklearn_dss_90,400
autosklearn_dss_90_mtl,400
autosklearn_dss_95,400
autosklearn_mtl,400
random_forest,400


In [193]:
from scipy.stats import wilcoxon

aux = df[["method", "dataset", "test"]]\
  .groupby(["method", "dataset"])\
  .agg(mean_test=("test", "mean"), std_test=("test", "std"))\
  .reset_index()\
  .pivot(index="dataset", columns="method", values=["mean_test", "std_test"])\
  .round(4)

wins = {
    'autosklearn': 0,
    'autosklearn_mtl': 0,
    'autosklearn_dss':0,
    'autosklearn_dss_90':0,
    'autosklearn_dss_90_mtl':0,
    'autosklearn_dss_95':0,
    'random_forest':0
}

result = []

for r in aux.iterrows():

  p_value = 1
  idx_g1 = (df["dataset"] == r[0]) & (df["method"] == "autosklearn_dss_90_mtl")
  group1 = df["test"][idx_g1]
  idx_g2 = (df["dataset"] == r[0]) & (df["method"] == "autosklearn_mtl")
  group2 = df["test"][idx_g2]
  idx_g3 = (df["dataset"] == r[0]) & (df["method"] == "random_forest")
  group3 = df["test"][idx_g3]

  stat2, p_value2 = wilcoxon(group1, group2)
  stat3, p_value3 = wilcoxon(group1, group3)

  marker2 = "\markersquare" if p_value2 <= 0.05 else None
  marker3 = "\markercircle" if p_value3 <= 0.05 else None

  g1_g2 = r[1]['mean_test']['autosklearn_dss_90_mtl'] > r[1]['mean_test']['autosklearn_mtl']
  g1_g3 = r[1]['mean_test']['autosklearn_dss_90_mtl'] > r[1]['mean_test']['random_forest']

  g1_higher_g2 = marker2 if g1_g2 and marker2 else ""
  g2_higher_g1 = marker2 if g1_g2 == False and marker2 else ""
  g1_higher_g3 = marker3 if g1_g3 and marker3 else ""
  g3_higher_g1 = marker3 if g1_g3 == False and marker3 else ""


  result.append({
      "dataset ID": int(r[0].split("_")[-1]),
      "autosklearn-mtl-dss-90": f"${round(r[1]['mean_test']['autosklearn_dss_90_mtl'],4):.4f} \pm {round(r[1]['std_test']['autosklearn_dss_90_mtl'], 2):.2f}$ {g1_higher_g2}{g1_higher_g3}",
      "autosklearn-mtl": f"${round(r[1]['mean_test']['autosklearn_mtl'],4):.4f} \pm {round(r[1]['std_test']['autosklearn_mtl'], 2):.2f}$ {g2_higher_g1}",
      "random-forest": f"${round(r[1]['mean_test']['random_forest'],4):.4f} \pm {round(r[1]['std_test']['random_forest'], 2):.2f}$ {g3_higher_g1}",
      "p-value vs autosklearn-mtl": f"{p_value2:.2f}",
      "p-value vs random-forest": f"{p_value3:.2f}"
  })

df_result_stat_table = pd.DataFrame(result).sort_values(by="dataset ID")
df_result_stat_table.to_csv("performance-table-3600.csv", index=False)
df_result_stat_table

/usr/local/lib/python3.11/dist-packages/scipy/stats/_wilcoxon.py:172: RuntimeWarning: invalid value encountered in scalar divide
  z = (r_plus - mn) / se


,dataset ID,autosklearn-mtl-dss-90,autosklearn-mtl,random-forest,p-value vs autosklearn-mtl,p-value vs random-forest
3,11,$0.9792 \pm 0.02$ \markercircle,$0.9980 \pm 0.01$ \markersquare,$0.9069 \pm 0.02$,0.01,0.00
15,23,$0.5471 \pm 0.02$,$0.5547 \pm 0.02$,$0.5425 \pm 0.02$,0.32,0.56
16,28,$0.9896 \pm 0.00$ \markersquare\markercircle,$0.9883 \pm 0.00$,$0.9807 \pm 0.00$,0.00,0.00
17,30,$0.9724 \pm 0.00$,$0.9720 \pm 0.01$,$0.9720 \pm 0.01$,0.70,0.62
12,155,$0.7661 \pm 0.24$,$0.9989 \pm 0.00$ \markersquare,$0.9926 \pm 0.00$,0.00,0.19
14,181,$0.6018 \pm 0.02$,$0.6003 \pm 0.02$,$0.5986 \pm 0.02$,0.62,0.62
18,307,$0.9729 \pm 0.02$ \markercircle,$0.9745 \pm 0.01$,$0.9313 \pm 0.01$,0.65,0.00
27,725,$0.9409 \pm 0.00$ \markercircle,$0.9432 \pm 0.00$ \markersquare,$0.9370 \pm 0.00$,0.02,0.02
28,761,$0.9372 \pm 0.01$ \markercircle,$0.9375 \pm 0.01$,$0.9332 \pm 0.00$,1.00,0.02
29,770,$1.0000 \pm 0.00$ \markercircle,$1.0000 \pm 0.00$,$0.9618 \pm 0.02$,1.00,0.00


In [179]:
wins

{'autosklearn': 0,
 'autosklearn_mtl': 0,
 'autosklearn_dss': 0,
 'autosklearn_dss_90': 0,
 'autosklearn_dss_90_mtl': 21,
 'autosklearn_dss_95': 0,
 'random_forest': 0}

In [137]:
df_stat_test = df[["seed", "dataset", "method",  "test"]]\
.pivot(index=["dataset", "seed"], columns=["method"], values="test")
df_stat_test

method             autosklearn  autosklearn_dss_90  autosklearn_dss_90_mtl  \
dataset      seed                                                            
dataset_1044 0        0.773813            0.660201                0.752757   
             1        0.738648            0.802483                0.736809   
             2        0.720948            0.743519                0.733044   
             3        0.750045            0.795137                0.747043   
             4        0.779248            0.780775                0.769327   
...                        ...                 ...                     ...   
dataset_951  5        1.000000            1.000000                1.000000   
             6        1.000000            1.000000                1.000000   
             7        1.000000            1.000000                1.000000   
             8        1.000000            1.000000                1.000000   
             9        1.000000            1.000000                1.000000   

method             autosklearn_dss_95  autosklearn_mtl  random_forest  
dataset      seed                                                      
dataset_1044 0               0.704437         0.754950       0.671331  
             1               0.722022         0.737543       0.682607  
             2               0.712816         0.720722       0.677942  
             3               0.733709         0.739743       0.694025  
             4               0.726550         0.771309       0.691461  
...                               ...              ...            ...  
dataset_951  5               1.000000         1.000000       1.000000  
             6               1.000000         1.000000       1.000000  
             7               1.000000         1.000000       1.000000  
             8               1.000000         1.000000       1.000000  
             9               1.000000         1.000000       1.000000  

[400 rows x 6 columns]

In [138]:
df_stat_test.to_csv("autosklearn_dss_v1.csv", index=False)

In [58]:
df_stat_test[["autosklearn_dss_90_mtl", "autosklearn_mtl", "random_forest"]].to_csv("autosklearn_dss_v2.csv", index=False)

In [120]:
df_aux = df\
  .assign(train_val=df["train"]-df["val"])\
  .assign(train_test=df["train"]-df["test"])\
  .assign(val_test=df["val"]-df["test"])
  # .groupby(["method", "dataset"])\
  # .agg(train_val=("train_val", "mean"), train_test=("train_test", "mean"), val_test=("val_test", "mean"))

df_aux

,train,val,test,method,dataset,seed,train_val,train_test,val_test
0,0.990755,0.975541,0.951557,autosklearn_mtl,dataset_934,0,0.015214,0.039198,0.023984
0,0.983882,0.982626,0.941339,autosklearn_mtl,dataset_934,1,0.001256,0.042543,0.041287
0,0.990790,0.975541,0.950685,autosklearn_mtl,dataset_934,2,0.015249,0.040105,0.024856
0,0.990773,0.982626,0.921477,autosklearn_mtl,dataset_934,4,0.008146,0.069296,0.061149
0,0.986133,0.972125,0.969110,autosklearn_mtl,dataset_934,3,0.014008,0.017023,0.003015
...,...,...,...,...,...,...,...,...,...
0,0.986143,0.970324,0.943411,autosklearn_dss_90_mtl,dataset_1166,5,0.015819,0.042732,0.026913
0,0.983569,0.960305,0.932817,autosklearn_dss_90_mtl,dataset_1166,6,0.023264,0.050752,0.027488
0,0.984982,0.956615,0.939774,autosklearn_dss_90_mtl,dataset_1166,7,0.028367,0.045208,0.016841
0,0.980947,0.952583,0.956257,autosklearn_dss_90_mtl,dataset_1166,8,0.028363,0.024689,-0.003674


In [144]:
df_tran_val_text_diff = df_aux\
  .groupby(["method"])\
  .agg(
      mean_train_val=("train_val", "mean"),
      mean_train_test=("train_test", "mean"),
      mean_val_test=("val_test", "mean"),
      std_train_val=("train_val", "std"),
      std_train_test=("train_test", "std"),
      std_val_test=("val_test", "std"),
  )\
  .round(4)\
  .query("method == 'autosklearn_dss_90_mtl' or  method == 'autosklearn_mtl'")
df_tran_val_text_diff

,mean_train_val,mean_train_test,mean_val_test,std_train_val,std_train_test,std_val_test
method,,,,,,
autosklearn_dss_90_mtl,0.0595,0.0844,0.0249,0.0948,0.1078,0.0378
autosklearn_mtl,0.0549,0.0790,0.0241,0.0892,0.1030,0.0376


In [155]:
result = []
for r in df_tran_val_text_diff.iterrows():
  result.append({
      "method": r[0],
      "train - test": f"${round(r[1]['mean_train_test'],4):.4f} \pm {round(r[1]['std_train_test'], 2):.2f}$",
      "train - val": f"${round(r[1]['mean_train_val'],4):.4f} \pm {round(r[1]['std_train_val'], 2):.2f}$",
      "val - test": f"${round(r[1]['mean_val_test'],4):.4f} \pm {round(r[1]['std_val_test'], 2):.2f}$"
  })

pd.DataFrame(result)

,method,train - test,train - val,val - test
0,autosklearn_dss_90_mtl,$0.0844 \pm 0.11$,$0.0595 \pm 0.09$,$0.0249 \pm 0.04$
1,autosklearn_mtl,$0.0790 \pm 0.10$,$0.0549 \pm 0.09$,$0.0241 \pm 0.04$


In [139]:
s, p = wilcoxon(df_aux.query("method == 'autosklearn_mtl'").train_test, df_aux.query("method == 'autosklearn_dss_90_mtl'").train_test)
p

np.float64(0.4430786377695357)

In [140]:
s, p = wilcoxon(df_aux.query("method == 'autosklearn_mtl'").train_val, df_aux.query("method == 'autosklearn_dss_90_mtl'").train_val)
p

np.float64(0.6978961034895343)

In [141]:
s, p = wilcoxon(df_aux.query("method == 'autosklearn_mtl'").val_test, df_aux.query("method == 'autosklearn_dss_90_mtl'").val_test)
p

np.float64(0.5071345087896635)